# Proyecto de Clasificación de Email

En el siguiente proyecto de ejemplo, se utilizará un algoritmo de aprendizaje de máquina supervisado, particularmente regresión logística, para determinar si un email es spam o no.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

## Data Collection & Pre-Processing

In [2]:
# loading the data from csv file to a pandas df
raw_mail_data = pd.read_csv("./data/mail_data.csv")
print(raw_mail_data.shape)
raw_mail_data.head()

(5572, 2)


,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
# no hay nulos, pero si los hubiera, se hace un tratamiento de los mismos
raw_mail_data[raw_mail_data["Message"].isnull()] 

,Category,Message


In [4]:
# replace the null values with a empty string
mail_data = raw_mail_data.where(pd.notnull(raw_mail_data), '')

In [5]:
# printing the first five rows
mail_data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
# checking the number of ows and columns
mail_data.shape

(5572, 2)

## Label Encoding

In [7]:
# label spam mail as 0 and non-spam mail as 1
mail_data["Category"] = mail_data["Category"].replace({"ham": 1, "spam": 0}).astype(int)
mail_data.head()

/var/folders/ph/5sq6jlj161g1_2t838crnx6c0000gp/T/ipykernel_2790/3673332426.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mail_data["Category"] = mail_data["Category"].replace({"ham": 1, "spam": 0}).astype(int)


,Category,Message
0,1,"Go until jurong point, crazy.. Available only ..."
1,1,Ok lar... Joking wif u oni...
2,0,Free entry in 2 a wkly comp to win FA Cup fina...
3,1,U dun say so early hor... U c already then say...
4,1,"Nah I don't think he goes to usf, he lives aro..."


In [8]:
# separating the data as text and labels
X = mail_data["Message"]

y = mail_data["Category"]

In [9]:
X.head()

0    Go until jurong point, crazy.. Available only ...
1                        Ok lar... Joking wif u oni...
2    Free entry in 2 a wkly comp to win FA Cup fina...
3    U dun say so early hor... U c already then say...
4    Nah I don't think he goes to usf, he lives aro...
Name: Message, dtype: object

In [10]:
y.head()

0    1
1    1
2    0
3    1
4    1
Name: Category, dtype: int64

## Splitting the data into training and test data

El parámetro `random_state` en la función `train_test_split` de scikit-learn es utilizado para controlar la aleatoriedad en la división de los datos en conjuntos de entrenamiento y prueba.

### Explicación:
Cuando usas train_test_split, esta función divide aleatoriamente el conjunto de datos en dos partes (usualmente un conjunto de entrenamiento y otro de prueba). El parámetro random_state asegura que la división de los datos sea reproducible.

* Si random_state tiene un valor fijo (por ejemplo, un número entero como 42), la división de los datos será siempre la misma cada vez que se ejecute el código, incluso si se ejecuta varias veces.

* Si random_state es None (valor predeterminado), entonces la división será aleatoria cada vez que se ejecute el código. Esto puede ser útil si deseas obtener diferentes divisiones en cada ejecución.

### ¿Por qué usarlo?
Usar random_state garantiza que puedas obtener la misma división de datos en ejecuciones repetidas. Esto es importante cuando estás probando diferentes modelos o realizando validaciones, ya que te permite comparar los resultados de manera consistente con el mismo conjunto de entrenamiento y prueba.

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

In [12]:
print(X.shape)
print(X_train.shape)
print(X_test.shape)

(5572,)
(4457,)
(1115,)


## Feature Extraction

### ¿Qué es `TfidfVectorizer`?

El **`TfidfVectorizer`** es una herramienta de **scikit-learn** utilizada para convertir un conjunto de textos (documentos) en una **representación numérica**. Utiliza el modelo **TF-IDF (Term Frequency-Inverse Document Frequency)** para transformar los textos en vectores de características que los algoritmos de aprendizaje automático pueden entender.

### ¿Qué es **TF-IDF**?

**TF-IDF** es una técnica para evaluar la importancia de una palabra en un conjunto de documentos (en este caso, un documento para nosotros sería una fila u observación del dataset). Se calcula utilizando dos componentes principales:

1. **TF (Term Frequency)**:
   - **Frecuencia del término** en un documento.
   - Mide cuántas veces aparece una palabra en un documento en comparación con el total de palabras en ese documento.
   - Fórmula:  
$\text{TF}(t) = \frac{\text{Número de veces que aparece el término } t}{\text{Número total de términos en el documento}}$

2. **IDF (Inverse Document Frequency)**:
   - **Frecuencia inversa de documentos**.
   - Mide cuán importante es una palabra en el conjunto de documentos, penalizando las palabras que aparecen en muchos documentos.
   - Fórmula:  
$\text{IDF}(t) = \log\left(\frac{\text{Número total de documentos}}{\text{Número de documentos que contienen el término } t}\right)$

La fórmula final de **TF-IDF** es:

$\text{TF-IDF}(t) = \text{TF}(t) \times \text{IDF}(t)$

El resultado es un valor que indica la importancia de una palabra en un documento, teniendo en cuenta no solo su frecuencia en el documento, sino también en cuántos documentos aparece en total.

---

### ¿Cómo funciona **`TfidfVectorizer`**?

1. **Entrada**:
   Recibe un conjunto de textos o documentos (generalmente una lista de cadenas de texto).

2. **Transformación**:
   Transforma los textos en una **matriz dispersa** de características numéricas, donde cada columna corresponde a una palabra única en el conjunto de documentos y cada fila corresponde a un documento.

3. **Salida**:
   La salida es una **matriz de características** donde cada valor representa el **TF-IDF** de una palabra en un documento.

---

### Ejemplo de uso:

```python
from sklearn.feature_extraction.text import TfidfVectorizer

# Conjunto de documentos
documentos = [
    "El gato come pescado",
    "El perro come huesos",
    "El gato duerme en la cama"
]

# Inicializar el TfidfVectorizer
vectorizer = TfidfVectorizer()

# Ajustar y transformar los documentos en una matriz TF-IDF
X = vectorizer.fit_transform(documentos)

# Ver las características (palabras)
print("Características:", vectorizer.get_feature_names_out())

# Ver la matriz de características (TF-IDF)
print("Matriz TF-IDF:\n", X.toarray())


En este ejemplo en particular, estamos creando nuestro vectorizador con tres parámetros:

* `min_df = 1`: se considerarán las palabras que aparecen en al menos un documento (es decir, no se filtrarán palabras que no aparezcan en ningún documento).
* `stop_words = 'english'`: descartará aquellas palabras que no tengan ningún significado en inglés.
* `lowercase = True`: transformará todas las palabras a minúsculas para que sea más fácil su posterior análisis para pasarlo a valor numérico.

In [13]:
# transform the text data to feature vectors and numerical values which can be used as input
# to Logistic Regression model
feature_extraction = TfidfVectorizer(min_df=1, stop_words='english', lowercase=True, norm=None)

X_train_features = feature_extraction.fit_transform(X_train)
X_test_features = feature_extraction.transform(X_test)

y_train = y_train.astype('int')
y_test = y_test.astype('int')

In [14]:
print(feature_extraction.get_feature_names_out())
print(len(feature_extraction.get_feature_names_out()))

['00' '000' '000pes' ... 'èn' 'ú1' '〨ud']
7431


In [15]:
print(X_train_features.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [16]:
print(X_train_features)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 34775 stored elements and shape (4457, 7431)>
  Coords	Values
  (0, 2329)	4.5821739483407775
  (0, 3811)	4.109150689221321
  (0, 2224)	4.880666936896774
  (0, 4456)	4.925118699467608
  (0, 5413)	7.323013972265978
  (1, 3811)	4.109150689221321
  (1, 3046)	5.905947952479334
  (1, 1991)	7.793017601511714
  (1, 2956)	7.793017601511714
  (1, 2758)	7.610696044717759
  (1, 1839)	6.569242169889598
  (1, 918)	5.395122328713343
  (1, 2746)	8.016161152825923
  (1, 2957)	8.016161152825923
  (1, 3325)	7.456545364890501
  (1, 3185)	7.004560241147444
  (1, 4080)	4.453695623567645
  (2, 6601)	8.70930833338587
  (2, 2404)	6.512083756049649
  (2, 3156)	5.905947952479334
  (2, 407)	7.323013972265978
  (3, 7414)	5.79153760130159
  (3, 2870)	4.192969361104393
  (4, 2870)	12.57890808331318
  (4, 487)	8.70930833338587
  :	:
  (4454, 2855)	8.70930833338587
  (4454, 2246)	8.70930833338587
  (4455, 4456)	4.925118699467608
  (4455, 3922)	6.18357968907

## Training the ML model with Logistic Regression

In [17]:
model = LogisticRegression(fit_intercept=True)
model.fit(X_train_features, y_train)

LogisticRegression()

## Evaluating the Model

In [18]:
# prediction on training data
y_train_predicted = model.predict(X_train_features)
print(accuracy_score(y_train, y_train_predicted))

1.0


In [19]:
# prediction on test data
y_test_predicted = model.predict(X_test_features)
print(accuracy_score(y_test, y_test_predicted))

0.9757847533632287


## Building a Predictive System

In [23]:
input_mail = ["I've been searching for the right words to thank you for this breather. I promise i wont take your help for granted and will fulfil my promise. You have been wonderful and a blessing at all times.",
              "You have won a 1 week FREE membership in our £100,000 Prize Jackpot! Txt the word: CLAIM to No: 81010 T&C www.dbuk.net",
              "Your account has benn hacked. Give us your password to recover access",
              "Dear User, Congratulations! You've been selected to receive a $1,000 Gift Card as part of our exclusive giveaway. To claim your reward, simply click the link below and complete the quick verification process: Claim Your Gift Card Now Hurry, this offer is valid for the next 24 hours only! Best regards, The Rewards Team"]

# convert text to feature vector with numerical values
input_mail_features = feature_extraction.transform(input_mail)

# making predictions
predictions = model.predict(input_mail_features)

print(predictions)

[1 0 1 0]


In [21]:
for prediction in predictions:
    if(prediction):
        print("No es spam")
    else:
        print("Spam")

No es spam
Spam
Spam


## Exporting Model

In [44]:
from sklearn2pmml import PMMLPipeline
from sklearn2pmml import sklearn2pmml

# Crear un pipeline con el vectorizador y el modelo
pipeline = PMMLPipeline([
    ("tfidf", feature_extraction),  # Vectorizador que ya definiste
    ("classifier", model)          # Modelo que ya entrenaste
])

# Exportar el modelo a PMML
sklearn2pmml(pipeline, "email_classifier.pmml", with_repr=True)
print("Modelo exportado como email_classifier.pmml")

Modelo exportado como email_classifier.pmml
